# DonorsChoose

In [1]:
!jupyter nbconvert --to html vamshinallagonda@gmail_A4_NBT.ipynb

[NbConvertApp] Converting notebook vamshinallagonda@gmail_A4_NBT.ipynb to html
[NbConvertApp] Writing 613089 bytes to vamshinallagonda@gmail_A4_NBT.html


DonorsChoose.org receives hundreds of thousands of project proposals each year for classroom projects in need of funding. Right
now, a large number of volunteers is needed to manually screen each submission before it's approved to be posted on the
DonorsChoose.org website.

Next year, DonorsChoose.org expects to receive close to 500,000 project proposals. As a result, there are three main problems they
need to solve:
    
 How to scale current manual processes and resources to screen 500,000 projects so that they can be posted as quickly and as
 efficiently as possible
 How to increase the consistency of project vetting across different volunteers to improve the experience for teachers
 How to focus volunteer time on the applications that need the most assistance
    
The goal of the competition is to predict whether or not a DonorsChoose.org project proposal submitted by a teacher will be
approved, using the text of project descriptions as well as additional metadata about the project, teacher, and school.
DonorsChoose.org can then use this information to identify projects most likely to need further review before approval.

In [2]:
import warnings 
warnings.filterwarnings("ignore")


In [3]:

import sqlite3
import pandas as pd
import numpy as np
import nltk
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc
from nltk.stem.porter import PorterStemmer
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn import cross_validation
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
import pickle
from sklearn.preprocessing import Normalizer
from tqdm import tqdm
import os
import math
from sklearn.externals import joblib

C:\Users\vamshi goud\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\vamshi goud\Anaconda3\lib\site-packages\smart_open\ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [4]:
#project_data = pd.read_csv('train_data.csv',nrows=50000)
data = pd.read_csv('train_data.csv',nrows=20000)
data2 = pd.read_csv('resources.csv')
data3 =data2.groupby('id').agg({'price':'sum','quantity':'sum'}).reset_index()
project_data = pd.merge(data,data3,on='id',how='left')
project_data['price'] = project_data['price'].fillna((project_data['price'].mean()))
project_data['quantity'] = project_data['quantity'].fillna((project_data['quantity'].mean()))
y = np.array(project_data['project_is_approved'])
x = project_data.drop('project_is_approved',axis=1)
xTrain,xTest,yTrain,yTest = train_test_split(x,y, test_size = 0.2, random_state =42)
xTrain_c,x_CV,yTrain_c,y_CV   = train_test_split(xTrain,yTrain, test_size = 0.2, random_state =42)
print(xTrain.shape)
print(xTest.shape)
print(yTrain.shape)
print(yTest.shape)
print(xTrain_c.shape)
print(x_CV.shape)
print(yTrain_c.shape)
print(y_CV.shape)

(16000, 18)
(4000, 18)
(16000,)
(4000,)
(12800, 18)
(3200, 18)
(12800,)
(3200,)


In [5]:
from sklearn.preprocessing import Imputer,OneHotEncoder
df = x
imputer = Imputer(missing_values='NaN',strategy='mean',axis=0)

df.head(5)

,Unnamed: 0,id,teacher_id,teacher_prefix,school_state,project_submitted_datetime,project_grade_category,project_subject_categories,project_subject_subcategories,project_title,project_essay_1,project_essay_2,project_essay_3,project_essay_4,project_resource_summary,teacher_number_of_previously_posted_projects,price,quantity
0,160221,p253737,c90749f5d961ff158d4b4d1e7dc665fc,Mrs.,IN,2016-12-05 13:43:57,Grades PreK-2,Literacy & Language,"ESL, Literacy",Educational Support for English Learners at Home,My students are English learners that are work...,"\""The limits of your language are the limits o...",NaN,NaN,My students need opportunities to practice beg...,0,154.60,23
1,140945,p258326,897464ce9ddc600bced1151f324dd63a,Mr.,FL,2016-10-25 09:22:10,Grades 6-8,"History & Civics, Health & Sports","Civics & Government, Team Sports",Wanted: Projector for Hungry Learners,Our students arrive to our school eager to lea...,The projector we need for our school is very c...,NaN,NaN,My students need a projector to help with view...,7,299.00,1
2,21895,p182444,3465aaf82da834c0582ebd0ef8040ca0,Ms.,AZ,2016-08-31 12:03:56,Grades 6-8,Health & Sports,"Health & Wellness, Team Sports",Soccer Equipment for AWESOME Middle School Stu...,"\r\n\""True champions aren't always the ones th...",The students on the campus come to school know...,NaN,NaN,"My students need shine guards, athletic socks,...",1,516.85,22
3,45,p246581,f3cb9bffbba169bef1a77b243e620b60,Mrs.,KY,2016-10-06 21:16:17,Grades PreK-2,"Literacy & Language, Math & Science","Literacy, Mathematics",Techie Kindergarteners,I work at a unique school filled with both ESL...,My students live in high poverty conditions wi...,NaN,NaN,My students need to engage in Reading and Math...,4,232.90,4
4,172407,p104768,be1f7507a41f8479dc06f047086a39ec,Mrs.,TX,2016-07-11 01:10:09,Grades PreK-2,Math & Science,Mathematics,Interactive Math Tools,Our second grade classroom next year will be m...,"For many students, math is a subject that does...",NaN,NaN,My students need hands on practice in mathemat...,1,67.98,4


## Text preprocessing of training 'essay' data

In [6]:
#text preprocessing of training 'essay' data

xTrain["essay"] = xTrain["project_essay_1"].map(str) +\
                  xTrain["project_essay_2"].map(str) + \
                  xTrain["project_essay_3"].map(str) + \
                  xTrain["project_essay_4"].map(str)


In [7]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
sent = decontracted(xTrain['essay'].values[15000])

In [8]:
sent = sent.replace('\\r', ' ')
sent = sent.replace('\\"', ' ')
sent = sent.replace('\\n', ' ')
sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))
print(stopwords)

{'i', "doesn't", 'here', 'did', 'into', 'have', 'wouldn', 'he', 'than', 'any', 'won', 'didn', 'haven', 'before', 'more', 'his', 'had', 'in', 'nor', "aren't", 'and', 'of', 'so', 'that', "you'd", "don't", 'all', 'each', 'not', 'ain', 'ourselves', 'the', 'then', 'until', "weren't", "you're", 'its', 'what', 'hasn', 'doesn', 'him', "couldn't", 'your', 's', "mightn't", 'itself', 'myself', "shouldn't", 'hadn', "should've", 'you', 'her', 'am', "mustn't", 'through', 'other', 'doing', 'were', 'was', 'be', 'off', 'above', 'are', 't', 'it', 'further', 'where', 'out', 'isn', 'yours', "shan't", "you've", 'my', 'they', 'themselves', "that'll", 'aren', 'having', 'those', "hadn't", 'an', 'theirs', 'there', 'most', "haven't", 'will', 'me', 'who', 'now', 'been', 'once', 'hers', 'such', 'or', 'if', 'couldn', 'o', 'because', 'herself', 'during', 're', 'mustn', 'on', 'at', 'm', 'yourselves', 'd', 'from', 'whom', 'do', 'himself', 'same', 'needn', 'being', 'for', 'while', 'how', "isn't", 'their', 'between', '

In [9]:
from tqdm import tqdm
preprocessed_essays_x = []
# tqdm is for printing the status bar
for sentance in tqdm(xTrain['essay'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_essays_x.append(sent.lower().strip())

100%|██████████████████████████████████████████████████████████████████████████| 16000/16000 [00:06<00:00, 2488.98it/s]


In [10]:
xTrain_c["essay"] = xTrain_c["project_essay_1"].map(str) +\
                        xTrain_c["project_essay_2"].map(str) + \
                       xTrain_c ["project_essay_3"].map(str) + \
                       xTrain_c["project_essay_4"].map(str)


In [11]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
sent = decontracted(xTrain_c['essay'].values[5000])

In [12]:
sent = sent.replace('\\r', ' ')
sent = sent.replace('\\"', ' ')
sent = sent.replace('\\n', ' ')
sent = re.sub('[^A-Za-z0-9]+', ' ', sent)


In [13]:
from tqdm import tqdm
preprocessed_essays_xc = []
# tqdm is for printing the status bar
for sentance in tqdm(xTrain_c['essay'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_essays_xc.append(sent.lower().strip())

100%|██████████████████████████████████████████████████████████████████████████| 12800/12800 [00:04<00:00, 2700.25it/s]


In [14]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
sent = decontracted(xTrain['project_title'].values[1500])

In [15]:
sent = sent.replace('\\r', ' ')
sent = sent.replace('\\"', ' ')
sent = sent.replace('\\n', ' ')
sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
from tqdm import tqdm
preprocessed_title_xT = []
# tqdm is for printing the status bar
for sentance in tqdm(xTrain['project_title'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sen = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_title_xT.append(sent.lower().strip())

100%|█████████████████████████████████████████████████████████████████████████| 16000/16000 [00:00<00:00, 24478.81it/s]


In [16]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
sent = decontracted(xTrain_c['project_title'].values[1000])

In [17]:
sent = sent.replace('\\r', ' ')
sent = sent.replace('\\"', ' ')
sent = sent.replace('\\n', ' ')
sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
from tqdm import tqdm
preprocessed_title_xC = []
# tqdm is for printing the status bar
for sentance in tqdm(xTrain_c['project_title'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sen = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_title_xC.append(sent.lower().strip())

100%|█████████████████████████████████████████████████████████████████████████| 12800/12800 [00:00<00:00, 22748.36it/s]


## Preprocessing of test data

In [18]:
#preprocessing of test data
xTest["essay"] = xTest["project_essay_1"].map(str) +\
                        xTest["project_essay_2"].map(str) + \
                        xTest["project_essay_3"].map(str) + \
                        xTest["project_essay_4"].map(str)


In [19]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
sent = decontracted(xTest['essay'].values[2000])

In [20]:
from tqdm import tqdm
preprocessed_essays_xTest = []
# tqdm is for printing the status bar
for sentance in tqdm(xTest['essay'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_essays_xTest.append(sent.lower().strip())

100%|████████████████████████████████████████████████████████████████████████████| 4000/4000 [00:01<00:00, 2462.94it/s]


## Preporcessing of cross validate data

In [21]:
x_CV["essay"] = x_CV["project_essay_1"].map(str) +\
                        x_CV["project_essay_2"].map(str) + \
                       x_CV["project_essay_3"].map(str) + \
                        x_CV["project_essay_4"].map(str)

In [22]:
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
sent = decontracted(x_CV['essay'].values[2000])

In [23]:
from tqdm import tqdm
preprocessed_essays_xcvT = []
# tqdm is for printing the status bar
for sentance in tqdm(x_CV['essay'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_essays_xcvT.append(sent.lower().strip())
   

100%|████████████████████████████████████████████████████████████████████████████| 3200/3200 [00:01<00:00, 2293.57it/s]


In [24]:
import re
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
sent = decontracted(xTest['project_title'].values[3500])

In [25]:
import re
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
sent = decontracted(xTrain_c['project_title'].values[3000])

In [26]:
sent = sent.replace('\\r', ' ')
sent = sent.replace('\\"', ' ')
sent = sent.replace('\\n', ' ')
sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
from tqdm import tqdm
preprocessed_title_xt = []
# tqdm is for printing the status bar
for sentance in tqdm(xTest['project_title'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent1 = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_title_xt.append(sent1.lower().strip())

100%|███████████████████████████████████████████████████████████████████████████| 4000/4000 [00:00<00:00, 21634.90it/s]


In [27]:
sent = sent.replace('\\r', ' ')
sent = sent.replace('\\"', ' ')
sent = sent.replace('\\n', ' ')
sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
from tqdm import tqdm
preprocessed_title_xt_c = []
# tqdm is for printing the status bar
for sentance in tqdm(xTrain_c['project_title'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
    # https://gist.github.com/sebleier/554280
    sent1 = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_title_xt_c.append(sent1.lower().strip())

100%|█████████████████████████████████████████████████████████████████████████| 12800/12800 [00:00<00:00, 18454.35it/s]


In [28]:
import re
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
sent = decontracted(xTest['project_title'].values[3500])

In [29]:
sent = sent.replace('\\r', ' ')
sent = sent.replace('\\"', ' ')
sent = sent.replace('\\n', ' ')
sent = re.sub('[^A-Za-z0-9]+', ' ', sent)
from tqdm import tqdm
preprocessed_title_cvt = []
# tqdm is for printing the status bar
for sentance in tqdm(x_CV['project_title'].values):
    sent = decontracted(sentance)
    sent = sent.replace('\\r', ' ')
    sent = sent.replace('\\"', ' ')
    sent = sent.replace('\\n', ' ')
    sent = re.sub('[^A-Za-z0-9]+', ' ', sent1)
    # https://gist.github.com/sebleier/554280
    sent1_t = ' '.join(e for e in sent.split() if e not in stopwords)
    preprocessed_title_cvt.append(sent1.lower().strip())

100%|███████████████████████████████████████████████████████████████████████████| 3200/3200 [00:00<00:00, 25614.85it/s]


In [30]:
import re
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase
sent = decontracted(x_CV['project_title'].values[1000])

In [31]:
categories = list(x['project_subject_categories'].values)
# remove special characters from list of strings python:
#https://stackoverflow.com/a/47301924/4084039
# https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
# https://stackoverflow.com/questions/23669024/how-to-strip-a-specific-word-from-a-string
# https://stackoverflow.com/questions/8270092/remove-all-whitespace-in-a-string-in-python
cat_list = []
for i in categories:
    temp = ""
# consider we have text like this "Math & Science, Warmth, Care & Hunger"
for j in i.split(','): # it will split it in three parts ["Math & Science", "Warmth", "Care & H

    if 'The' in j.split(): # this will split each of the catogory based on space "Math & Scienc
        j=j.replace('The','') # if we have the words "The" we are going to replace it with ''(i
        j = j.replace(' ','') # we are placeing all the ' '(space) with ''(empty) ex:"Math &
    temp+=j.strip()+" " #" abc ".strip() will return "abc", remove the trailing spaces
    temp = temp.replace('&','_') # we are replacing the & value into
    cat_list.append(temp.strip())
   # print(cat_list.)
    #print(x.le)
#xTrain.apply(lambda col: col.drop_duplicates().reset_index(drop=True))
#xTrain('project_subject_categories') = cat_list
   # x['clean_categories'] = cat_list
    #project_data.drop(['project_subject_categories'], axis=1, inplace=True)
from collections import Counter
my_counter = Counter()
for word in x['project_subject_categories'].values:
    my_counter.update(word.split())
cat_dict = dict(my_counter)
sorted_cat_dict = dict(sorted(cat_dict.items(), key=lambda kv: kv[1]))
print(sorted_cat_dict)

{'Arts,': 34, 'Needs,': 57, 'Warmth,': 264, 'Care': 264, 'Hunger': 264, 'Civics,': 428, 'Sports,': 516, 'Civics': 629, 'History': 1057, 'Learning': 1070, 'Learning,': 1201, 'Science,': 1480, 'Arts': 1793, 'Music': 1827, 'The': 1827, 'Sports': 2104, 'Applied': 2271, 'Needs': 2447, 'Special': 2504, 'Health': 2620, 'Language,': 4005, 'Language': 5639, 'Science': 6054, 'Math': 7534, 'Literacy': 9644, '&': 22946}


In [32]:
sub_categories = list(project_data['project_subject_subcategories'].values)
# remove special characters from list of strings python:
#https://stackoverflow.com/a/47301924/4084039
# https://www.geeksforgeeks.org/removing-stop-words-nltk-python/
# https://stackoverflow.com/questions/23669024/how-to-strip-a-specific-word-from-a-string
# https://stackoverflow.com/questions/8270092/remove-all-whitespace-in-a-string-in-python
sub_cat_list = []
for i in sub_categories:
   

    temp = ""
# consider we have text like this "Math & Science, Warmth, Care & Hunger"
for j in i.split(','): # it will split it in three parts ["Math & Science", "Warmth", "Care & H
    if 'The' in j.split(): # this will split each of the catogory based on space "Math & Scienc
        j=j.replace('The','') # if we have the words "The" we are going to replace it with ''(
    j = j.replace(' ','') # we are placeing all the ' '(space) with ''(empty) ex:"Math &
    temp +=j.strip()+" "#" abc ".strip() will return "abc", remove the trailing spaces
    temp = temp.replace('&','_')
    sub_cat_list.append(temp.strip())
#project_data['clean_subcategories'] = sub_cat_list
#project_data.drop(['project_subject_subcategories'], axis=1, inplace=True)
# count of all the words in corpus python: https://stackoverflow.com/a/22898595/4084039
my_counter = Counter()
for word in project_data['project_subject_subcategories'].values:
    my_counter.update(word.split())
sub_cat_dict = dict(my_counter)
sorted_sub_cat_dict = dict(sorted(sub_cat_dict.items(), key=lambda kv: kv[1]))

## Vectorization of essay 

In [33]:
vectorizer_tfid = TfidfVectorizer()
xtrain_tfidf_essay = vectorizer_tfid.fit(preprocessed_essays_x)
xtrain_tfidf_essay = vectorizer_tfid.transform(preprocessed_essays_x)
xtrain_tfidf_essay_cv = vectorizer_tfid.transform(preprocessed_essays_xc)
xtest_tfidf_esssay = vectorizer_tfid.transform(preprocessed_essays_xTest)
xtest_tfidf_esssay_cv = vectorizer_tfid.transform(preprocessed_essays_xcvT)

## vectorization of title 

In [34]:
vectorizer_tfid = TfidfVectorizer()
xtrain_tfidf_title = vectorizer_tfid.fit(preprocessed_title_xT)
xtrain_tfidf_title =vectorizer_tfid.transform(preprocessed_title_xT)
xtrain_tfidf_title_cv = vectorizer_tfid.transform(preprocessed_title_xC)
xtest_tfidf_title =vectorizer_tfid.transform(preprocessed_title_xt)
xtest_tfidf_title_cv = vectorizer_tfid.transform(preprocessed_title_cvt)

## vectorization of categorical data

In [35]:

vectorizer = CountVectorizer(min_df=10,max_features=5000)
vectorizer.fit(xTrain['project_subject_categories'].values)
categories_one_hot_train = vectorizer.transform(xTrain['project_subject_categories'].values)
categories_one_hot_test = vectorizer.transform(xTest['project_subject_categories'].values)
categories_one_hot_cv = vectorizer.transform(xTrain_c['project_subject_categories'].values)

In [36]:
vectorizer = CountVectorizer(min_df=10,max_features=5000)
vectorizer.fit(xTrain['project_subject_subcategories'].values)
sub_categories_one_hot_train = vectorizer.transform(xTrain['project_subject_subcategories'].values)
sub_categories_one_hot_test = vectorizer.transform(xTest['project_subject_subcategories'].values)
sub_categories_one_hot_cv = vectorizer.transform(xTrain_c['project_subject_subcategories'].values)

In [37]:
my_counter = Counter()
for state in project_data['school_state'].values:
    my_counter.update(state.split())
school_state_dict = dict(my_counter)
sorted_school_state_dict = dict(sorted(school_state_dict.items(), key=lambda kv: kv[1]))

In [38]:
vectorizer = CountVectorizer(min_df=10,max_features=5000)
vectorizer.fit(xTrain['school_state'].values)
school_state_categories_one_hot_train = vectorizer.transform(xTrain['school_state'].values)
school_state_categories_one_hot_test = vectorizer.transform(xTest['school_state'].values)
school_state_categories_one_hot_cv = vectorizer.transform(xTrain_c['school_state'].values)

In [39]:
my_counter = Counter()
for project_grade in project_data['project_grade_category'].values:
    my_counter.update(project_grade.split())
project_grade_dict = dict(my_counter)
sorted_project_grade_dict = dict(sorted(project_grade_dict.items(), key=lambda kv: kv[1]))

In [40]:
vectorizer = CountVectorizer(min_df=10,max_features=5000)
vectorizer.fit(xTrain['project_grade_category'].values)
project_grade_categories_one_hot_train = vectorizer.transform(xTrain['project_grade_category'].values)
project_grade_categories_one_hot_test =vectorizer.transform(xTest['project_grade_category'].values)
project_grade_categories_one_hot_cv = vectorizer.transform(xTrain_c['project_grade_category'].values)
#print(project_grade_categories_one_hot_train)

In [41]:
my_counter = Counter()
for teacher_prefix in project_data['teacher_prefix'].values:
    teacher_prefix = str(teacher_prefix)
    my_counter.update(teacher_prefix.split())
teacher_prefix_dict = dict(my_counter)
sorted_teacher_prefix_dict = dict(sorted(teacher_prefix_dict.items(), key=lambda kv: kv[1]))

In [42]:
## https://stackoverflow.com/questions/39303912/tfidfvectorizer-in-scikit-learn-valueerror-np-nanis-an-invalid-document/39308809#39308809
vectorizer = CountVectorizer(min_df = 10,max_features = 5000)
vectorizer.fit(xTrain['teacher_prefix'].values.astype("U"))
teacher_prefix_categories_one_hot_train =vectorizer.transform(xTrain['teacher_prefix'].values.astype("U"))
teacher_prefix_categories_one_hot_test =vectorizer.transform(xTest['teacher_prefix'].values.astype("U"))
teacher_prefix_categories_one_hot_cv =vectorizer.transform(xTrain_c['teacher_prefix'].values.astype("U"))
print(vectorizer.get_feature_names())

['mr', 'mrs', 'ms', 'teacher']


In [43]:
normalizer = Normalizer()
normalizer.fit(xTrain['teacher_number_of_previously_posted_projects'].values.reshape(1,-1))
prev_projects_train = normalizer.transform(xTrain['teacher_number_of_previously_posted_projects'].values.reshape(1,-1))
#rev_projects_train1 = normalizer.transform(xTrain['teacher_number_of_previously_posted_projects'].values.reshape(-1,1))
prev_projects_cv = normalizer.transform(xTrain_c['teacher_number_of_previously_posted_projects'].values.reshape(1,-1))
prev_projects_test = normalizer.transform(xTest['teacher_number_of_previously_posted_projects'].values.reshape(1,-1))
print("After vectorizations")
prev_projects_train = prev_projects_train.reshape(-1,1)
prev_projects_cv = prev_projects_cv.reshape(-1,1)
prev_projects_test= prev_projects_test.reshape(-1,1)
print(prev_projects_train)
#print(prev_projects_train1)
print(prev_projects_train.shape, yTrain.shape)
print(prev_projects_cv.shape, y_CV.shape)
print(prev_projects_test.shape, yTest.shape)

After vectorizations
[[0.00157365]
 [0.00550778]
 [0.00026228]
 ...
 [0.0010491 ]
 [0.        ]
 [0.00052455]]
(16000, 1) (16000,)
(12800, 1) (3200,)
(4000, 1) (4000,)


In [44]:
normalizer = Normalizer()
normalizer.fit(xTrain['quantity'].values.reshape(1,-1))
quant_projects_train = normalizer.transform(xTrain['quantity'].values.reshape(1,-1))
quant_projects_cv = normalizer.transform(xTrain_c['quantity'].values.reshape(1,-1))
quant_projects_test = normalizer.transform(xTest['quantity'].values.reshape(1,-1))
print(quant_projects_train)
quant_projects_train = quant_projects_train.reshape(-1,1)
quant_projects_cv = quant_projects_cv.reshape(-1,1)
quant_projects_test = quant_projects_test.reshape(-1,1)

[[0.00406679 0.00127087 0.00686271 ... 0.01270871 0.01321706 0.00177922]]


In [45]:
normalizer = Normalizer()
normalizer.fit(xTrain['price'].values.reshape(1,-1))
price_projects_train = normalizer.transform(xTrain['price'].values.reshape(1,-1))
price_projects_cv = normalizer.transform(xTrain_c['price'].values.reshape(1,-1))
price_projects_test = normalizer.transform(xTest['price'].values.reshape(1,-1))
print(price_projects_train)
price_projects_train = price_projects_train.reshape(-1,1)
price_projects_cv = price_projects_cv.reshape(-1,1)
price_projects_test = price_projects_test.reshape(-1,1)

[[0.00268353 0.00243484 0.00451044 ... 0.00036411 0.00020681 0.01051077]]


In [46]:
#vectorizing the text data
vectorizer = CountVectorizer(min_df=10,max_features=5000)
xtrain_bow_essay = vectorizer.fit(preprocessed_essays_x)
xtrain_bow_essay = vectorizer.transform(preprocessed_essays_x)
#xTrain_bow_essay_c = vectorizer.fit(preprocessed_essays_xc)
xtrain_bow_essay_c = vectorizer.transform(preprocessed_essays_xc)
xtest_bow_esssay = vectorizer.transform(preprocessed_essays_xTest)
xtest_bow_esssay_cv = vectorizer.transform(preprocessed_essays_xcvT)
print(xtest_bow_esssay.shape)

(4000, 5000)


In [47]:
#vectorzing the title text data
vectorizer = CountVectorizer(min_df=10,max_features=5000)
xtrain_bow_title = vectorizer.fit(preprocessed_title_xT)
xtrain_bow_title =vectorizer.transform(preprocessed_title_xT)
#xTrain_bow_title_c = vectorizer.fit(preprocessed_title_xt_c)
xtrain_bow_title_c = vectorizer.transform(preprocessed_title_xt_c)
xtest_bow_title =vectorizer.transform(preprocessed_title_xt)
xtest_bow_title_cv = vectorizer.transform(preprocessed_title_cvt)

## stacking all features  of  training data with bow

In [48]:
from scipy.sparse import hstack
features=hstack((prev_projects_train,project_grade_categories_one_hot_train,categories_one_hot_train,teacher_prefix_categories_one_hot_train,sub_categories_one_hot_train,xtrain_bow_essay,xtrain_bow_title,quant_projects_train,price_projects_train)).tocsr()
features = features.toarray()
print(features.shape)

(16000, 6010)


## stacking all features of cross validate data with bow

In [49]:
features_bow = hstack((quant_projects_cv,price_projects_cv,prev_projects_cv,project_grade_categories_one_hot_cv,categories_one_hot_cv,teacher_prefix_categories_one_hot_cv,sub_categories_one_hot_cv,xtrain_bow_essay_c,xtrain_bow_title_c))
features_bow = features_bow.toarray()
print(features_bow.shape)

(12800, 6010)


## stacking all features  of  test data with bow

In [50]:
xTest_features = hstack((quant_projects_test,price_projects_test,prev_projects_test,project_grade_categories_one_hot_test,categories_one_hot_test,teacher_prefix_categories_one_hot_test,sub_categories_one_hot_test,xtest_bow_title,xtest_bow_esssay)).tocsr()
xTest_features = xTest_features.toarray()
print(xTest_features.shape)

(4000, 6010)


In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
train_auc = []
cv_auc = []
kl = np.log([10**x for x in range (-4,4)])


neigh = MultinomialNB(class_prior=[0.5,0.5])

#clf = MultinomialNB(alpha=[10*2,2)], class_prior=None, fit_prior=True)
    # roc_auc_score(y_true, y_score) the 2nd parameter should be probability estimates of the positive class
    # not the predicted outputs
parameters = {'alpha':k}   
clf = GridSearchCV(neigh,param_grid = parameters, cv=3, scoring='roc_auc')
clf.fit(features, yTrain)
filename = 'finalized_model.sav'
pickle.dump(clf, open(filename,'wb'))



In [ ]:
loaded_model = pickle.load(open(filename,'rb'))
y_train_pred = loaded_model .predict_proba(features)[:,1]
y_cv_pred =  loaded_model.predict_proba(features_bow)[:,1]
    
train_auc.append(roc_auc_score(yTrain,y_train_pred))
cv_auc.append(roc_auc_score(yTrain_c, y_cv_pred))
train_auc= loaded_model.cv_results_['mean_train_score']
train_auc_std= loaded_model.cv_results_['std_train_score']
cv_auc = loaded_model.cv_results_['mean_test_score'] 
cv_auc_std= loaded_model.cv_results_['std_test_score']

kl = np.log([10**x for x in range (-4,4)])

plt.plot(kl, train_auc, label='Train AUC')
#this code is copied from here: https://stackoverflow.com/a/48803361/4084039

plt.gca().fill_between(kl,train_auc - train_auc_std,train_auc + train_auc_std,alpha=0.2,color='darkblue')
plt.plot(kl, cv_auc, label='CV AUC')
plt.gca().fill_between(kl,cv_auc - cv_auc_std,cv_auc + cv_auc_std,alpha=0.2,color='darkorange')
plt.legend()
plt.xlabel("K: hyperparameter")
plt.ylabel("AUC")
plt.title("ERROR PLOTS")
plt.grid()
plt.show()  


In [ ]:
##https://stackoverflow.com/questions/47582264/python-how-to-convert-a-list-to-loglist
kl = np.log([10**x for x in range (-4,4)])
print(kl)

In [ ]:
print(loaded_model .best_score_)
print(loaded_model .best_params_)
print(loaded_model .best_estimator_)
y_test_pred = loaded_model .predict_proba(xTest_features)[:,1]

## Observations

1) By Observing cross validate data point in above plot,the maximunm best score is 0.68            
2) For that maximum score the best hyperparameter i,e alpha is 0.1 for multinominal NB

In [ ]:
from sklearn.naive_bayes import MultinomialNB
neigh = MultinomialNB(alpha=0.1, class_prior=[0.5,0.5], fit_prior=True)
final1 = neigh.fit(features, yTrain)
final = neigh.predict_proba(xTest_features)
print(final)
train_fpr, train_tpr, tr_thresholds = roc_curve(yTrain, y_train_pred)
test_tpr, test_fpr, te_thresholds = roc_curve(yTest, y_test_pred)
plt.plot(train_fpr, train_tpr, label="Train AUC ="+str(auc(train_fpr, train_tpr)))
plt.plot(test_fpr, test_tpr, label="Test AUC ="+str(auc(test_fpr, test_tpr)))
plt.legend()
plt.xlabel("True Positive Rate(TPR)")
plt.ylabel("False Positive Rate(FPR)")
plt.title("AUC")
plt.grid()
plt.show()

## Observations

1) By  observing above plot area under curve (AUC),is above 0.60,it means model performing better,beacuse its above average score

In [ ]:
from sklearn.metrics import confusion_matrix
# we are writing our own function for predict, with defined thresould
# we will pick a threshold that will give the least fpr
def find_best_threshold(threshould, fpr, tpr):
    t = threshould[np.argmax(tpr*(1-fpr))]
    # (tpr*(1-fpr)) will be maximum if your fpr is very low and tpr is very high
    print("the maximum value of tpr*(1-fpr)", max(tpr*(1-fpr)), "for threshold", np.round(t,3))
    return t

def predict_with_best_t(proba, threshould):
    predictions = []
    for i in proba:
        if i>=threshould:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions
print("="*100)
from sklearn.metrics import confusion_matrix
best_t = find_best_threshold(tr_thresholds, train_fpr, train_tpr)
print("Train confusion matrix")
print(confusion_matrix(yTrain, predict_with_best_t(y_train_pred, best_t)))
print("Test confusion matrix")
print(confusion_matrix(yTest, predict_with_best_t(y_test_pred, best_t)))

## Observations

1) From above confution matrix,we can say that ,the model predicted 2540 as true positive and 14429 as true negitive in train data

2) In test data it predicted 952 as true postive and 110 as true neagative,as per plot it predicts 60%

In [ ]:
import pandas as pd
heatmap_matrix =pd.DataFrame(confusion_matrix(yTrain, predict_with_best_t(y_train_pred, best_t)) , columns=["actual values:NO",'actual value:YES'])
print(heatmap_matrix)
sns.heatmap(heatmap_matrix,annot=True, fmt='g',xticklabels='auto',yticklabels='auto')  
plt.show()


In [ ]:
import pandas as pd
heatmap_matrix_t =pd.DataFrame(confusion_matrix(yTest, predict_with_best_t(y_test_pred, best_t)),columns=["actual values:NO",'actual value:YES'])
print(heatmap_matrix_t)
sns.set()
sns.heatmap(heatmap_matrix_t,annot=True, fmt='g')  
plt.show()

In [ ]:
from scipy.sparse import hstack
features_tfid=hstack((price_projects_train,quant_projects_train,prev_projects_train,project_grade_categories_one_hot_train,categories_one_hot_train,teacher_prefix_categories_one_hot_train,sub_categories_one_hot_train,xtrain_tfidf_essay,xtrain_tfidf_title)).tocsr()
features_tfid = features_tfid.toarray()
print(features_tfid.shape)

In [ ]:
features_cv_tfid = hstack((price_projects_cv,quant_projects_cv,prev_projects_cv,project_grade_categories_one_hot_cv,categories_one_hot_cv,teacher_prefix_categories_one_hot_cv,sub_categories_one_hot_cv,xtrain_tfidf_essay_cv,xtrain_tfidf_title_cv))


In [ ]:
features_cv_tfid = features_cv_tfid.toarray()
print(features_cv_tfid.shape)

In [ ]:
xTest_features_tfid = hstack((price_projects_test,quant_projects_test,prev_projects_test,project_grade_categories_one_hot_test,categories_one_hot_test,teacher_prefix_categories_one_hot_test,sub_categories_one_hot_test,xtest_tfidf_esssay,xtest_tfidf_title)).tocsr()
xTest_features_tfid = xTest_features_tfid.toarray()
print(xTest_features_tfid.shape)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt

#k=[10**x for x in range (-4,4)]
kl = np.log([10**x for x in range (-4,4)])
#alphas=10*(-2,2)

neigh = MultinomialNB()
#clf.fit(features, yTrain)
#clf = MultinomialNB(alpha=[10*(-2,2)], class_prior=None, fit_prior=True)
    # roc_auc_score(y_true, y_score) the 2nd parameter should be probability estimates of the positive class
    # not the predicted outputs
parameters = {'alpha':k1}   
clf1 = GridSearchCV(neigh,param_grid = parameters, cv=3, scoring='roc_auc')
clf1.fit(features_tfid, yTrain)
filename = 'finalized_model.sav'
#model, open(filename, 'wb')
pickle.dump(clf1, open(filename,'wb'))



In [ ]:
train_auc1 = []
cv_auc1 = []
loaded_model1 = pickle.load(open(filename, 'rb'))
y_train_pred_tfid =  loaded_model1.predict_proba(features_tfid)[:,1]
y_cv_pred_tfid =  loaded_model1.predict_proba(features_cv_tfid)[:,1]
    
train_auc1.append(roc_auc_score(yTrain,y_train_pred_tfid))
cv_auc1.append(roc_auc_score(yTrain_c, y_cv_pred_tfid))
train_auc= loaded_model1.cv_results_['mean_train_score']
train_auc_std= loaded_model1.cv_results_['std_train_score']
cv_auc = loaded_model1.cv_results_['mean_test_score'] 
cv_auc_std= loaded_model1.cv_results_['std_test_score']

plt.plot(kl, train_auc, label='Train AUC')
plt.gca().fill_between(k1,train_auc - train_auc_std,train_auc + train_auc_std,alpha=0.2,color='darkblue')
plt.plot(kl, cv_auc, label='CV AUC')
plt.gca().fill_between(k1,cv_auc - cv_auc_std,cv_auc + cv_auc_std,alpha=0.2,color='darkorange')
plt.legend()
plt.xlabel("K: hyperparameter")
plt.ylabel("AUC")
plt.title("ERROR PLOTS")
plt.grid()
plt.show()  

In [ ]:
print(loaded_model1.best_score_)
print(loaded_model1.best_params_)
print(loaded_model1.best_estimator_)
y_test_pred_tfid = loaded_model1.predict_proba(xTest_features_tfid)[:,1]

In [ ]:
from sklearn.naive_bayes import MultinomialNB
neigh = MultinomialNB(alpha=0.1, class_prior=None, fit_prior=True)
final1 = neigh.fit(features_tfid, yTrain)
final = neigh.predict_proba(xTest_features_tfid)
print(final)
train_fpr1, train_tpr1, tr_thresholds1 = roc_curve(yTrain, y_train_pred_tfid)
test_fpr1, test_tpr1, te_thresholds1 = roc_curve(yTest, y_test_pred_tfid)
plt.plot(train_fpr1, train_tpr1, label="Train AUC ="+str(auc(train_fpr1, train_tpr1)))
plt.plot(test_fpr1, test_tpr1, label="Test AUC ="+str(auc(test_fpr1, test_tpr1)))
plt.legend()
plt.xlabel("True Positive Rate(TPR)")
plt.ylabel("False Positive Rate(FPR)")
plt.title("AUC")
plt.grid()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
# we are writing our own function for predict, with defined thresould
# we will pick a threshold that will give the least fpr
def find_best_threshold(threshould, fpr, tpr):
    t = threshould[np.argmax(tpr*(1-fpr))]
    # (tpr*(1-fpr)) will be maximum if your fpr is very low and tpr is very high
    print("the maximum value of tpr*(1-fpr)", max(tpr*(1-fpr)), "for threshold", np.round(t,3))
    return t

def predict_with_best_t(proba, threshould):
    predictions = []
    for i in proba:
        if i>=threshould:
            predictions.append(1)
        else:
            predictions.append(0)
    return predictions
print("="*100)
from sklearn.metrics import confusion_matrix
#best_t = find_best_threshold(tr_thresholds, train_fpr1, train_tpr1)
print("Train confusion matrix")
print(confusion_matrix(yTrain, predict_with_best_t(y_train_pred_tfid, best_t)))
print("Test confusion matrix")
print(confusion_matrix(yTest, predict_with_best_t(y_test_pred_tfid, best_t)))


In [ ]:
import pandas as pd
heatmap_matrix =pd.DataFrame(confusion_matrix(yTrain, predict_with_best_t(y_train_pred_tfid, best_t)),columns=["actual values:NO",'actual value:YES'])
print(heatmap_matrix)
sns.heatmap(heatmap_matrix,annot=True, fmt='g')  
plt.show()

In [ ]:
import pandas as pd
heatmap_matrix =pd.DataFrame(confusion_matrix(yTrain, predict_with_best_t(y_train_pred_tfid, best_t)),columns=["actual values:NO",'actual value:YES'])
print(heatmap_matrix)
sns.heatmap(heatmap_matrix,annot=True, fmt='g')  
plt.show()

In [ ]:
from prettytable import PrettyTable
#If you get a ModuleNotFoundError error , install prettytable using: pip3 install prettytable
x = PrettyTable()
x.field_names = ["Vectorizer", "Model", "Hyper Parameter", "AUC"]
x.add_row(["BOW", "Brute", 1, 0.49])
x.add_row(["TFIDF", "Brute", 1, 0.60])

print(x)